In [1]:
import trafilatura
import requests
import lxml
from elasticsearch import Elasticsearch, helpers


In [2]:
kenh = {
    "life": ["https://kenh14.vn/xem-an-choi.chn",
             "https://kenh14.vn/doi-song/nhan-vat.chn",
             "https://kenh14.vn/doi-song/tram-yeu.chn"],
    "news": ["https://kenh14.vn/xa-hoi/song-xanh.chn",
             "https://kenh14.vn/xa-hoi/phap-luat.chn",
             "https://kenh14.vn/xa-hoi/nong-tren-mang.chn"],
    "world": ["https://kenh14.vn/the-gioi/di.chn",
              "https://kenh14.vn/the-gioi/chum-anh.chn",
              "https://kenh14.vn/the-gioi/kham-pha.chn"]
}


In [3]:
kenhLinks = {
    "life": [],
    "news": [],
    "world": []
}
for key, value in kenh.items():
    for url in value:
        response = requests.get(url)
        tree = lxml.html.fromstring(response.content)
        title_elem = tree.cssselect('.knswli-title a')
        for elem in title_elem:
            kenhLinks[key].append("https://kenh14.vn"+elem.get('href'))

print(kenhLinks)

{'life': ['https://kenh14.vn/snack-khoai-tay-lays-chinh-phuc-trai-tim-gioi-tre-viet-voi-nguon-khoai-tay-noi-dia-20220404170545249.chn', 'https://kenh14.vn/ra-mat-chuong-trinh-truyen-hinh-thuc-te-du-hanh-tuoi-tho-20220404170414634.chn', 'https://kenh14.vn/dat-them-30-thoa-thuan-moi-marriott-international-du-kien-tang-gap-4-lan-so-khach-san-va-khu-nghi-duong-tai-viet-nam-20220404145814664.chn', 'https://kenh14.vn/lo-dien-dien-mao-moi-cua-ctm-palace-an-tuong-va-gay-choang-ngop-hiem-co-tai-ha-noi-20220402153558058.chn', 'https://kenh14.vn/siberian-wellness-huong-toi-su-phat-trien-cua-the-he-tre-20220402103437425.chn', 'https://kenh14.vn/hang-ngan-voucher-uu-dai-cua-bamboo-airways-da-co-chu-trong-nhung-ngay-dau-vitm-2022-20220402122301579.chn', 'https://kenh14.vn/co-nen-luu-tru-te-bao-goc-cho-be-20220401220852111.chn', 'https://kenh14.vn/gai-dep-duoc-menh-danh-than-tien-ty-ty-dang-di-thi-hoa-hau-la-sinh-vien-luat-biet-3-thu-tieng-20220326172258659.chn', 'https://kenh14.vn/32-tuoi-chua-chong

In [4]:
kenhContent = {
    "life": [],
    "news": [],
    "world": []
}
for key, value in kenhLinks.items():
    for url in value:
        if(url.startswith('https://kenh14.vn/')):
            id = url.split('-')[-1].replace('.chn', '')
            downloaded = trafilatura.fetch_url(url)
            tree = lxml.html.fromstring(str(downloaded))
            date_elem = len(tree.cssselect(
                ".kbwcm-time")) and tree.cssselect(".kbwcm-time")[0].text or ""
            content = trafilatura.extract(downloaded)
            content = content.replace('\n', ' ')
            item = {"domain": "https://kenh14.vn/", "URL": url,
                    "content": content, "date": date_elem, "id": id}
            kenhContent[key].append(item)

print(len(kenhContent['life'])+len(kenhContent['news'])+len(kenhContent['world']))
print(kenhContent['life'][0])

20220404170545249
20220404170414634
20220404145814664
20220402153558058
20220402103437425
20220402122301579
20220401220852111
20220326172258659
20220321131927068


KeyboardInterrupt: 

In [ ]:
client = Elasticsearch("http://localhost:9200")

client.info()


In [5]:
def gendata():
    for key, value in kenhContent.items():
        for item in value:
            yield {
                "_index": "journal",
                "_type": key,
                "_id": item['id'],
                "doc": item
            }


helpers.bulk(es, gendata())


NameError: name 'helpers' is not defined